# Imports

In [ ]:
!pip install benepar

     |████████████████████████████████| 81kB 5.9MB/s 
  Created wheel for benepar: filename=benepar-0.1.2-cp36-cp36m-linux_x86_64.whl size=104452 sha256=bdd8e2b1af62119305f2680feaf73cab10786945f3251151b1dc3bbabede3ed8
  Stored in directory: /root/.cache/pip/wheels/c6/f5/06/d88543b19a9b326007d7538298a139e994b1d2eecb003bf5af
Successfully built benepar


In [ ]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 31kB/s 
     |████████████████████████████████| 368kB 44.7MB/s 
     |████████████████████████████████| 3.2MB 56.8MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# NLTK to find word stems
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

# Berkeley Neural Parser
# https://github.com/nikitakit/self-attentive-parser
import benepar
#benepar.download('benepar_en2')


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

# Data Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive/Vision\ and\ Language\ Project/

'BDD100k Experiment.ipynb'	   Proposal.gdoc
'BDD100k Exploration.ipynb'	   revisedBDDX.csv
 BDD100kExplore2.ipynb		   revisedBDDX.gsheet
 det_v2_train_release.json	  'Scenario2Vec Overview.gslides'
 Links.gdoc			   sdl_objects.npy
 MyFile.txt			   SDL_working.ipynb
'Naive to Bert Overview.gslides'   videos


In [ ]:
%cd drive/My\ Drive/Vision\ and\ Language\ Project/

/content/drive/My Drive/Vision and Language Project


In [ ]:
import pandas as pd # Pandas library enables data manipulation
data_url = "revisedBDDX.csv"
def load_bddx_data(csv_name):
    column_names = ['Index', 'InputVideo', '1S', '1E', '1A', '1J', '2S', '2E', '2A', '2J', '3S', '3E', '3A', '3J',
                    '4S', '4E', '4A', '4J','5S', '5E', '5A', '5J','6S', '6E', '6A', '6J','7S', '7E', '7A', '7J',
                    '8S', '8E', '8A', '8J','9S', '9E', '9A', '9J','10S', '10E', '10A', '10J','11S', '11E', '11A', '11J',
                    '12S', '12E', '12A', '12J','13S', '13E', '13A', '13J','14S', '14E', '14A', '14J','15S', '15E', '15A', '15J']
    
    return pd.read_csv(csv_name, names=column_names)
bddx = load_bddx_data(data_url)

bddx.drop(bddx.index[:1], inplace=True)

check = 10.0
for i,row in bddx.iterrows():
  start = 2
  while row[start] != 'NaN' and start+4 <= 61:
    if float(row[start]) < check and float(row[start+1]) > check:
      start = start + 4
    else:      
      bddx[row.index[start]].loc[i] = ''
      bddx[row.index[start+1]].loc[i] = ''
      bddx[row.index[start+2]].loc[i] = 'NaN'
      bddx[row.index[start+3]].loc[i] = ''
      start = start + 4


bddx = bddx.drop(['1S', '1E','2S', '2E','3S', '3E','4S', '4E','5S', '5E','6S', '6E','7S', '7E','8S', '8E','9S', '9E','10S', '10E','11S', '11E','12S', '12E','13S', '13E','14S', '14E','15S', '15E', ], axis=1)
bddx = bddx.fillna("")

bddx['1AJ'] = bddx[['1A', '1J']].agg(' '.join, axis=1)
bddx['2AJ'] = bddx[['2A', '2J']].agg(' '.join, axis=1)
bddx['3AJ'] = bddx[['3A', '3J']].agg(' '.join, axis=1)
bddx['4AJ'] = bddx[['4A', '4J']].agg(' '.join, axis=1)
bddx['5AJ'] = bddx[['5A', '5J']].agg(' '.join, axis=1)
bddx['6AJ'] = bddx[['6A', '6J']].agg(' '.join, axis=1)
bddx['7AJ'] = bddx[['7A', '7J']].agg(' '.join, axis=1)
bddx['8AJ'] = bddx[['8A', '8J']].agg(' '.join, axis=1)
bddx['9AJ'] = bddx[['9A', '9J']].agg(' '.join, axis=1)
bddx['10AJ'] = bddx[['10A', '10J']].agg(' '.join, axis=1)
bddx['11AJ'] = bddx[['11A', '11J']].agg(' '.join, axis=1)
bddx['12AJ'] = bddx[['12A', '12J']].agg(' '.join, axis=1)
bddx['13AJ'] = bddx[['13A', '13J']].agg(' '.join, axis=1)
bddx['14AJ'] = bddx[['14A', '14J']].agg(' '.join, axis=1)
bddx['15AJ'] = bddx[['15A', '15J']].agg(' '.join, axis=1)

bddx = bddx.drop(['Index', '1A', '1J', '2A', '2J', '3A', '3J', '4A', '4J', '5A', '5J', '6A', '6J', '7A', '7J', '8A', '8J', '9A', '9J', '10A', '10J', '11A', '11J', '12A', '12J', '13A', '13J', '14A', '14J', '15A', '15J', ], axis=1)
# bddx = bddx.drop(bddx.index[0])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
li = []
for i in bddx['InputVideo']:
  l = i.split("/")
  li.append(l[-1].split(".")[0])
print(li)
bddx['BDDX-ids'] =np.array(li)

['06d501fd-a9ffc960', '01b0505f-5f564e84', '06d501fd-fd237e38', '06d54ae6-26a3446e', '01b4e4b9-e21fe0a3', '06d54ae6-76749641', '01b4e4b9-fc7a5264', '06d54ae6-bf7ed032', '06d981bc-01758867', '01c3c73e-4279ee42', '06d981bc-d8411e3b', '01c3e19a-fbacb266', '0336de82-d8b4761a', '0004a4c0-d4df3a18', '06db23de-8789538f', '01c6a5ce-7dba0d93', '01d25bbc-670b6caa', '06e4a00e-346390c8', '00091078-2bcd1ac9', '05266828-79a6ed22', '03370efc-c690f4d8', '01d25bbc-a9f903ba', '06e4a00e-9ed56776', '00091078-38ea90aa', '0339a42b-1e75ac9a', '01d373c7-45710346', '05266828-a6ee9545', '06e4a00e-b0887752', '06ea71d8-6f0a4db0', '0526fe78-ad2264aa', '00091078-59817bb0', '01d7e66d-8d639849', '033dea13-5a8114fe', '06ebf6cd-0c129f12', '00091078-7cff8ea6', '052fd7cc-b5dee1de', '0340d277-0531808b', '01dc1bbe-8919b3d8', '06ebf6cd-79235c8d', '00091078-84635cf2', '0344b71d-0063c015', '0531d2f7-1feee72c', '06ec79fd-3351771d', '0348a36b-deb80913', '00091078-875c1f73', '053da4e3-48ec49ba', '06ec79fd-7b557fdf', '00091078-87

In [ ]:
bddx

,InputVideo,1AJ,2AJ,3AJ,4AJ,5AJ,6AJ,7AJ,8AJ,9AJ,10AJ,11AJ,12AJ,13AJ,14AJ,15AJ,BDDX-ids
1,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car accelerates because the light has turn...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,06d501fd-a9ffc960
2,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car is stopped. The car is at an intersect...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,01b0505f-5f564e84
3,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car accelerates slowly to a maintained spe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,06d501fd-fd237e38
4,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,06d54ae6-26a3446e
5,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car is stopped because the light is red.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,01b4e4b9-e21fe0a3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car is driving slowly since it is getting ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2c6484bb-396e2db8
6993,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car is driving forward The car is driving ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2c6484bb-5396d8e2
6994,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,NaN,The car is driving down the road The car has c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2c6484bb-8e0c070c
6995,https://s3-us-west-2.amazonaws.com/sq8geewpqu/...,The car is stopped. Traffic ahead of the car i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2c66430a-0dfc3a40


# SDL Class

In [ ]:
class Actor:
    def __init__(self, description):
        """
        self.description should be one of the following:
          ego
          light vehicle
          heavy vehicle
          cyclist
          pedestrian
        """
        self.description = description
        self.action = ""

In [ ]:
import nltk
benepar.download('benepar_en2')
class SDL_Util:
    def __init__(self):
        self.stemmer = nltk.stem.PorterStemmer()
        # stemmer allows us to match words with the same roots: e.g., it identifies "turns", "turning", and "turn" as the same word
        # But it recognizes "slow" and "slowly" as different, and it thinks "go" and "goes" are different
        self.parser = benepar.Parser("benepar_en2")
        # allows us to turn sentences into parse trees

        self.actor_list = {'car':'light vehicle',
                           'bus':'heavy vehicle',
                           'truck':'heavy vehicle',
                           'cyclist':'cyclist',
                           'pedestrian':'pedestrian',
                           'ambulance':'heavy vehicle',
                           'minivan':'light vehicle',
                           'traffic':'traffic',
                           }
        self.scene_list = ['intersect', # intersection stem
                           'crosswalk',
                           'bridge',
                           'light',
                           'sign',
                           'stop sign',
                           'yield sign',
                           'traffic light',
                           'traffic signal',
                           'turn lane',
                           ]

        init_action_list = {'turn':'turn',
                            'merge':'merge',
                            'swerve':'merge',
                            'veer':'merge',
                            'switch':'merge',
                            'accelerate':'accelerate',
                            'pick':'accelerate',
                            'brake':'brake',
                            'slow':'brake',
                            'reduce':'brake',
                            'decelerate':'brake',
                            'stop':'stop',
                            'wait':'stop',
                            'sit':'stop',
                            'forward':'forward',
                            'move':'forward',
                            'stay':'forward',
                            'maintain':'forward',
                            'proceeds':'forward',
                            'proceed':'forward',
                            'inch':'forward',
                            'pass':'forward',
                            'roll':'forward',
                            'advance':'forward',
                            'drive':'drive',
                            'steer':'drive',
                            'go':'drive',
                            'goes':'drive',
                            'head':'drive',
                            'pull':'drive',
                            'travel':'drive',
                            'flow':'drive',
                            'reverse':'reverse',
                            'walk':'walk',
                            'cross':'walk',
                            'park':'park',
                            'drift':'forward', # Not sure about this one, but it's not changing lanes so I'm calling it "forward"
                            'block':'stop', # Vehicles that are blocking generally are stopped in the path of the ego
                            'enter':'merge',
                            'straighten':'forward',
                            'follow':'forward',
                            'shift':'merge',
                            'change':'merge',
                            'stand':'stop', # Standing still
                            'curve':'forward',
                           }
        self.adjective_action = {'clear':'_negative', # traffic is clear
                                 'stationary':'stop', # car is stationary
                                 'complete':'stop', # is at a complete stop
                                 'heavy':'forward', # traffic is heavy
                                 'accelerating':'accelerate', # car is accelerating
                                 'light':'_negative', # traffic is light
                                 'slow':'forward', # traffic is slow
                                 'parallel':'reverse', # car is parallel parking
                                 'rolling':'forward', # car is rolling forward
                                 'driving':'drive', # car is driving
                                 'full':'stop', # car is at a full stop
                                 'stopped':'stop', # car is stopped
                                 'busy':'forward', # traffic is busy
                                 'double':'stop', # car is double parked
                                 'double-parked':'stop', # car is double-parked
                                 'idle':'stop', # car is idle
                                 'stopping':'brake', # car is stopping
                                 'minimal':'_negative', # traffic is minimal
                                 'empty':'_negative', # traffic is empty
                                 'moderate':'forward', # traffic is moderate
                                }
        self.action_list = {}
        for a in init_action_list:
            self.action_list[self.stemmer.stem(a)] = init_action_list[a]

        self.light_status = ["green","yellow","red"]
        self.directions = ["left","right","u-turn","uturn","through","backward","down","straight"]

        self.debug = {"_None":[]}
        
        self.count = 0
    def update_debug(self,word,phrase):
        if not word in self.debug:
            self.debug[word] = []
        self.debug[word].append(phrase)

util = SDL_Util()


[nltk_data] Downloading package benepar_en2 to /root/nltk_data...


In [ ]:
class SDL:
    def __init__(self, index, statements, link):
        self.statements = statements
        self.index = index
        l = link.split("/")
        l = l[-1].split(".")[0]
        self.videoLink = l

        self.actors = {'1':[], '2':[], '3':[], '4':[], '5':[], '6':[], '7':[], 
                       '8':[], '9':[], '10':[], '11':[], '12':[], '13':[], '14':[], '15':[]}
        self.scene = {'1':[], '2':[], '3':[], '4':[], '5':[], '6':[], '7':[], 
                       '8':[], '9':[], '10':[], '11':[], '12':[], '13':[], '14':[], '15':[]}
                
    def getDescriptors(self, statement, timeSegment):

        if(statement == "No Data Recorded"):
            # No caption => no SDL
            self.actors[timeSegment].append(Actor("NaN"))
            self.actors[timeSegment][-1].action = "NaN"
            self.scene[timeSegment].append("NaN")
            return

        parse_tree = util.parser.parse(statement.lower()) # Create a parse tree        
        self.sbar2sdl(parse_tree,timeSegment) # Search for actors and actions
        
    def sbar2sdl(self, tree, timeSegment):
        if type(tree) == str or len(tree) == 0 or type(tree[0]) == str:
            return # tree is a leaf and not a tree
        vp = -1 # Keep track of where the verb phrase is
        np = -1 # Keep track of where the noun phrase is
        todo = [] # Keep track of sub clauses to recursively check for more SDL elements
        sentence_break = []
        for i,node in enumerate(tree):
            if node.label()[0] == "S":
                self.sbar2sdl(node, timeSegment)
            elif node.label() == "NP":
                if len(self.actors[timeSegment]) > 0 and node.leaves() == ['the','car'] and \
                i < len(tree)-1 and tree[i+1].label() != "PP":
                    continue # Ego has been referenced a second time. Ignore it.
                self.np2sdl(node, timeSegment)
            elif node.label() == "VP":
                self.vp2sdl(node, timeSegment)
            elif node.label() == "PP":
                self.sbar2sdl(node, timeSegment)
                            
    def np2sdl(self, tree, timeSegment, neg=False):
        noun = ""
        verb = ""
        adj = ""
        todo = []
        for i,node in enumerate(tree):
            if type(node) == str: # tree is a leaf and not a tree
                break
            if node.label()[0] == "N" and len(node.leaves()) == 1:
                # Found the noun in the noun phrase
                if neg: # Actor is NOT present (e.g., "no traffic")
                    neg = False
                    continue
                elif noun == "":
                    noun = node.leaves()[0]
                else: #elif noun != "":
                    noun = noun+" "+node.leaves()[0]
                if i > 0 and tree[i-1].label()[0] == "V" and len(tree[i-1].leaves()) == 1:
                    # Verb before the noun is an action
                    verb = tree[i-1].leaves()[0]
                elif i > 0 and tree[i-1].label()[0] == "J" and len(tree[i-1].leaves()) == 1:
                    adj = tree[i-1].leaves()[0]
            elif node.label()[0] == "S":
                todo.append((node,"S",None))
                neg = False
            elif node.label() == "NP":
                todo.append((node,"N",neg))
                neg = False
            elif node.label() == "VP":
                todo.append((node,"V",neg))
                neg = False
            elif node.label() == "PP":
                todo.append((node,"S",None))
                neg = False
        noun = util.stemmer.stem(noun)
        verb = util.stemmer.stem(verb)
        if noun in util.scene_list:
            # Noun was a scene, add it to the scene list
            if noun == "light" and adj in util.light_status:
                # Check if the modifier describes the color of the traffic light
                noun = adj+" "+noun
            self.scene[timeSegment].append(noun)
        elif noun in util.actor_list:
            if len(self.actors[timeSegment]) == 0:
                noun = "ego"
            else:
                noun = util.actor_list[noun]
            self.actors[timeSegment].append(Actor(noun))
            if verb != "" and verb in util.action_list:
                verb = util.action_list[verb]
                # Add action to the associated actor
                self.actors[timeSegment][-1].action = verb
        elif noun in util.directions and len(self.actors[timeSegment]) > 0 and \
        self.actors[timeSegment][-1].action == "":
            self.actors[timeSegment][-1].action = noun+" turn"
        elif noun in util.action_list and len(self.actors[timeSegment]) > 0 and \
        self.actors[timeSegment][-1].action == "":
            noun = util.action_list[noun]
            self.actors[timeSegment][-1].action = noun
        for node,label,pass_neg in todo:
            if label == "S":
                self.sbar2sdl(node, timeSegment)
            elif label == "N":
                self.np2sdl(node, timeSegment, pass_neg)
            elif label == "V":
                self.vp2sdl(node, timeSegment, pass_neg)
    
    def vp2sdl(self, tree, timeSegment, neg=False):
        verb = ""
        adj = ""
        adv = ""
        todo = []
        flip = False
        for i,node in enumerate(tree):
            if type(node) == str: # tree is a leaf and not a tree
                break
            if node.label()[0] == "V" and len(node.leaves()) == 1:
                verb = node.leaves()[0]
                if neg:
                    flip = True
                neg = False
            elif node.label() == "ADJP" and len(node.leaves()) == 1:
                adj = node.leaves()[0]
                neg = False
            elif node.label() == "ADVP" and len(node.leaves()) == 1:
                adv = node.leaves()[0]
                neg = False
            elif node.label() == "RB" and len(node.leaves()) == 1 and \
            node.leaves()[0] == "not" or node.leaves()[0] == "n't":
                neg = True
            elif node.label()[0] == "S":
                todo.append((node,"S",None))
                neg = False
            elif node.label() == "NP":
                todo.append((node,"N",neg))
                neg = False
            elif node.label() == "VP":
                todo.append((node,"V",neg))
                neg = False
            elif node.label() == "PP":
                pp_verb = False
                if len(node.leaves()) < 5 and len(node.leaves()) > 0:
                    pot_verb = node.leaves()[-1]
                    pot_verb = util.stemmer.stem(pot_verb)
                    if pot_verb in util.action_list:
                        verb = pot_verb
                        pp_verb = True
                if not pp_verb:
                    todo.append((node,"S",None))
                neg = False
        verb = util.stemmer.stem(verb)
        if adj in util.light_status and len(self.scene[timeSegment]) > 0 and \
        self.scene[timeSegment][-1] == "light":
            self.scene[timeSegment][-1] = adj+" light"
        if verb in util.action_list and len(self.actors[timeSegment]) > 0 and \
        self.actors[timeSegment][-1].action == "":
            verb = util.action_list[verb]
            if flip and (verb == "forward" or verb == "drive"):
                verb = util.action_list["stop"]
            elif flip and (verb == "stop"):
                verb = util.action_list["forward"]
            elif adv in util.directions:
                verb = adv+" "+verb
            self.actors[timeSegment][-1].action = verb
        for node,label,pass_neg in todo:
            if label == "S":
                self.sbar2sdl(node, timeSegment)
            elif label == "N":
                self.np2sdl(node, timeSegment, pass_neg)
            elif label == "V":
                self.vp2sdl(node, timeSegment, pass_neg)


In [ ]:
sdlList = []
for index, row in bddx.iterrows():
    sdlList.append(row.astype(str))
assert len(sdlList) == 6996, "length of sdl list should be 6996"

In [ ]:
sdlStatements = []
for i in range(len(sdlList)):
    sdlStatements.append({'1': sdlList[i]['1AJ'], '2': sdlList[i]['2AJ'], '3': sdlList[i]['3AJ'], '4': sdlList[i]['4AJ'], 
                          '5': sdlList[i]['5AJ'], '6': sdlList[i]['6AJ'], '7': sdlList[i]['7AJ'], '8': sdlList[i]['8AJ'], 
                          '9': sdlList[i]['9AJ'], '10': sdlList[i]['10AJ'], '11': sdlList[i]['11AJ'], '12': sdlList[i]['12AJ'], 
                          '13': sdlList[i]['13AJ'], '14': sdlList[i]['14AJ'], '15': sdlList[i]['15AJ']})

for i in range(len(sdlStatements)):
    keysTemp = list(sdlStatements[i].keys())
    for j in keysTemp:
        if (sdlStatements[i][j]==" "):
            sdlStatements[i][j] = "No Data Recorded"

sdlObjectList = []
for i in range(len(sdlList)):
    sdlObjectList.append(SDL(i, sdlStatements[i], sdlList[i]['InputVideo']))

In [ ]:
import nltk
nltk.download('punkt')
for i in range(len(sdlObjectList)):
    if i%100 == 0:
        print(i)
    keysTemp = list(sdlObjectList[i].statements.keys())
    for j in (keysTemp):
        sdlObjectList[i].getDescriptors(sdlObjectList[i].statements[j], j)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900


In [ ]:
examples = [1,5730,1999] 

for example in examples:
    print('Object %i: '%(example))
    print("Actors: ")
    print(sdlObjectList[example].statements)

    for a in range(len(sdlObjectList[example].actors)):
      actorsIndex = str(a+1)
      for j in range(len(sdlObjectList[example].actors[actorsIndex])):
        print('Time segment:', actorsIndex, ",  %s: %s"%(sdlObjectList[example].actors[actorsIndex][j].description,sdlObjectList[example].actors[actorsIndex][j].action))
    print('Scene: ', sdlObjectList[example].scene)

Object 1: 
Actors: 
{'1': 'The car is stopped. The car is at an intersection with a red light.', '2': 'NaN ', '3': 'NaN ', '4': 'NaN ', '5': 'NaN ', '6': 'NaN ', '7': 'NaN ', '8': 'NaN ', '9': 'NaN ', '10': 'NaN ', '11': 'NaN ', '12': 'NaN ', '13': 'NaN ', '14': 'NaN ', '15': 'No Data Recorded'}
Time segment: 1 ,  ego: stop
Time segment: 15 ,  NaN: NaN
Scene:  {'1': ['intersect', 'red light'], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': [], '10': [], '11': [], '12': [], '13': [], '14': [], '15': ['NaN']}
Object 5730: 
Actors: 
{'1': 'The car is stopped because there are cars moving along.', '2': 'NaN ', '3': 'NaN ', '4': 'NaN ', '5': 'NaN ', '6': 'NaN ', '7': 'NaN ', '8': 'NaN ', '9': 'NaN ', '10': 'NaN ', '11': 'NaN ', '12': 'NaN ', '13': 'NaN ', '14': 'NaN ', '15': 'No Data Recorded'}
Time segment: 1 ,  ego: stop
Time segment: 1 ,  light vehicle: forward
Time segment: 15 ,  NaN: NaN
Scene:  {'1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [

In [ ]:
# Save off sdl objects
import numpy as np
sdl_np = np.array(sdlObjectList)
np.save('sdl_objects', sdl_np)

In [ ]:
# Load sdl objects into array
sdlObjectList = np.load('sdl_objects.npy', allow_pickle=True)

In [ ]:
print(len(sdlObjectList))

6996


In [ ]:
examples = [0] 
print(bddx.index[6995])
for example in examples:
    print('Object %i: '%(example))
    print("Actors: ")
    print(sdlObjectList[example].statements)

    for a in range(len(sdlObjectList[example].actors)):
      actorsIndex = str(a+1)
      for j in range(len(sdlObjectList[example].actors[actorsIndex])):
        print('Time segment:', actorsIndex, ",  %s: %s"%(sdlObjectList[example].actors[actorsIndex][j].description,sdlObjectList[example].actors[actorsIndex][j].action))
    print('Scene: ', sdlObjectList[example].scene)
    scene_list = []
    for i in sdlObjectList[example].scene.values():
        if len(i) !=0:
            for x in i:
                scene_list.append(x)
    print(scene_list)


6996
Object 0: 
Actors: 
{'1': 'The car accelerates because the light has turned green.', '2': 'NaN ', '3': 'NaN ', '4': 'NaN ', '5': 'NaN ', '6': 'NaN ', '7': 'NaN ', '8': 'NaN ', '9': 'NaN ', '10': 'NaN ', '11': 'NaN ', '12': 'NaN ', '13': 'NaN ', '14': 'NaN ', '15': 'No Data Recorded'}
Time segment: 1 ,  ego: accelerate
Time segment: 15 ,  NaN: NaN
Scene:  {'1': ['green light'], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': [], '10': [], '11': [], '12': [], '13': [], '14': [], '15': ['NaN']}
['green light', 'NaN']


In [ ]:
# Count how many actors have associated actions
actor_count = 0
action_count = 0
for idx in range(len(sdlObjectList)):
    so = sdlObjectList[idx]
    for timestep in so.actors:
        for actor in so.actors[timestep]:
            if actor.description == "NaN":
                break
            actor_count+=1
            if actor.action != "":
                action_count+=1

print("There are %i actors and %i actions. Completeness: %f"%(actor_count,action_count,action_count/actor_count))

# Display the top 10 unclassified verbs
print("Top 10 unclassified verbs:")
dd = sorted(util.debug.items(), key = lambda kv:(-len(kv[1]),kv[0]))
cutoff = 10
for (key,value) in dd:
    print("  %s: %i"%(key,len(value)))
    cutoff-=1
    if cutoff <= 0:
        break

There are 8441 actors and 7373 actions. Completeness: 0.873475
Top 10 unclassified verbs:
  _None: 0


# SDL Representation with Matrices

In [ ]:
import numpy as np 
'''

For each SDL object in sdlObjectList, (2 x n) matrix represents the action-actor pair with n being the number of actor action pairs, (1 x m) matrix 
where m represents the number of scene elements

'''
actor_encoding = {'light vehicle': 0, 'heavy vehicle': 1, 'cyclist': 2, 'pedestrian': 3, 'traffic': 4, 'ego': 5, 'NaN': 6}

action_encoding = {'turn': 0, 'turn left': 1, 'turn right': 3, 'merge': 4, 'accelerate': 5, 'brake': 6, 'stop': 7, 
                   'forward': 8, 'walk': 9, 'park': 10, 'drive': 11, 'reverse': 12, 'merge center': 13, 'merge left': 14, 
                   'merge right': 15, 'turn through': 16, 'merge u turn': 17, 'u-turn': 18, 'NaN': 19, '':20}

scene_encoding = {'intersection': 0, 'crosswalk': 1, 'bridge': 2, 'green light': 3, 'stop sign': 4, 'yield sign': 5, 'sign': 6, 
                  'u-turn': 7, 'traffic light': 8, 'traffic signal': 9, 'turn lane': 10, 'crosswalks': 11, 'green traffic light': 12, 
                  'light': 13, 'lights': 14, 'red light': 15, 'red traffic light': 16, 'signs': 17, 'traffic lights': 18, 
                  'yellow light': 19, 'yellow traffic light': 20, 'NaN': 21}

sdl_embeddings = []

for example in range(len(sdlObjectList)): #loops through 6996 sdl objects in sdlObjectList
    # Each sdl object has a 7x21x22 one hot encoding representing its action, actor and scene element
    for a in range(len(sdlObjectList[example].actors)): #loops through 15 time segments
      actorsIndex = str(a+1)
      actor_list = []
      action_list = []
      scene_list = []
      for j in range(len(sdlObjectList[example].actors[actorsIndex])):
        actor_list.append(sdlObjectList[example].actors[actorsIndex][j].description)
        action_list.append(sdlObjectList[example].actors[actorsIndex][j].action)

      scene_list = sdlObjectList[example].scene[actorsIndex]
      
      # print("actor list: ", actor_list)
      # print("action list: ", action_list)
      # print("scene list: ", scene_list)

      actor_indices = []
      action_indices = []
      scene_indices = []

      if (len(actor_list) != len(action_list)):
        print("Actor and action list don't match up, this may cause 1 to 1 actor to action correspondence errors")
        break
      
      sdl_matrices = []
      actor_action_matrix = np.zeros((len(actor_list), 2))


      scene_length = 0
      if (len(scene_list) == 0):
          scene_length = 1
      else:
          scene_length = len(scene_list)    
      scene_matrix = np.zeros((scene_length,))


      for a_index in actor_list:
        actor_indices.append(actor_encoding[a_index])

      for act_index in action_list:
        action_indices.append(action_encoding[act_index])

      if ( ((len(scene_list)) > 0) and scene_list[0] != 'NaN'):
        for i in scene_list:
            scene_indices.append(scene_encoding[i])
      else:
        scene_indices.clear()
        scene_indices.append(21)

      if (len(actor_indices) != len(action_indices)):
        print("make sure each actor is matched up with an action")
        break

      for i, actor_index in enumerate(actor_indices):
          actor_action_matrix[i][0] = actor_index

      for j, action_index in enumerate(action_indices):
          actor_action_matrix[j][1] = action_index

      for k, scene_index in enumerate(scene_indices):
          scene_matrix[k] = scene_index

      sdl_matrices.append(actor_action_matrix)
      sdl_matrices.append(scene_matrix)
      sdl_embeddings.append(sdl_matrices)

assert len(sdl_embeddings) == 104940, "length of sdlEmbeddings should be 104940"

In [ ]:
sdl_embeddings[45][1]

array([21.])

# BDD100K Setup

## Bdd100k Data Cleaning/Intersection with Bddx

In [ ]:
bddx_ids = li

In [ ]:
import json
# path = "drive/My Drive/bdd100k/bdd100k_labels_images_train.json"
file_name = "det_v2_train_release.json"
with open(file_name) as f:
    data = json.load(f)

In [ ]:
print(len(data))
bdd100k_ids = []
for i in data:
    bdd100k_ids.append(i['videoName'])
bdd100k_no_repeat = list(dict.fromkeys(bdd100k_ids))

69863


In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 
intersection_list = intersection(bddx_ids, bdd100k_no_repeat)

In [ ]:
data_sub = []
for i, x in enumerate(data):
    if (x['videoName'] in intersection_list):
        data_sub.append(x)
print(len(data_sub) == len(intersection_list))

True


In [ ]:
print(data_sub[1]) #for that video name, find the corresponding timestamps, which interval 10,000 falls into, 10 seconds, see if 10000(ms) - see which interval this falls into in the BDDX dataset adn pick that up

{'videoName': '00091078-7cff8ea6', 'name': '00091078-7cff8ea6.jpg', 'index': 0, 'timestamp': 10000, 'labels': [{'category': 'car', 'poly2d': None, 'box3d': None, 'box2d': {'x2': 738.140616, 'y1': 288.4628777129832, 'x1': 276.517073, 'y2': 679.8129514206904}, 'manualShape': True, 'attributes': {'occluded': True, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'id': 204}, {'category': 'traffic sign', 'poly2d': None, 'box3d': None, 'box2d': {'x2': 289.153732, 'y1': 91.674132, 'x1': 127.702913, 'y2': 195.299926}, 'manualShape': True, 'attributes': {'occluded': False, 'truncated': False, 'trafficLightColor': [0, 'NA']}, 'id': 205}, {'category': 'car', 'poly2d': None, 'box3d': None, 'box2d': {'x2': 195.935553, 'y1': 359.099874, 'x1': 0, 'y2': 521.099819}, 'manualShape': True, 'attributes': {'occluded': True, 'truncated': True, 'trafficLightColor': [0, 'NA']}, 'id': 206}, {'category': 'car', 'poly2d': None, 'box3d': None, 'box2d': {'x2': 296.999895, 'y1': 389.699861, 'x1': 264.599906, 'y

In [ ]:
for i in data_sub:
    if ('index' in i):
        i.pop('index')
    if ('name' in i):
        i.pop('name')
for i in data_sub:
    if ('weather' in i['attributes']):
        i['attributes'].pop('weather')
    if ('timeofday' in i['attributes']):
        i['attributes'].pop('timeofday')
for i in data_sub:
    labels_list = i['labels']
    if labels_list:
        for x in labels_list:
            if ('poly2d' in x):
                x.pop('poly2d')
            if ('box3d' in x):
                x.pop('box3d')
            if ('box2d' in x):
                x.pop('box2d')
            if ('manualShape' in x):
                x.pop('manualShape')
            if ('score' in x):
                x.pop('score')
            if ('manualAttributes' in x):
                x.pop('manualAttributes')
            if ('id' in x):
                x.pop('id')
            if ('occluded' in x['attributes']):
                x['attributes'].pop('occluded')
            if ('truncated' in x['attributes']):
                x['attributes'].pop('truncated')
            if ('areaType' in x['attributes']):
                x['attributes'].pop('areaType')
            if ('laneDirection' in x['attributes']):
                x['attributes'].pop('laneDirection')
            if ('laneStyle' in x['attributes']):
                x['attributes'].pop('laneStyle')


In [ ]:
print("Video Name: ", data_sub[0]['videoName'])
print("Time stamp: ", data_sub[0]['timestamp'])
labels_list = (data_sub[5]['labels'])
for i, x in enumerate(labels_list):
    print("object: ", i, " category: ", x['category'], " attributes: ", x['attributes'])
print("Scene: ", data_sub[0]['attributes']['scene'])

Video Name:  00091078-59817bb0
Time stamp:  10000
object:  0  category:  traffic light  attributes:  {'trafficLightColor': [0, 'NA']}
object:  1  category:  pedestrian  attributes:  {'trafficLightColor': [0, 'NA']}
object:  2  category:  traffic light  attributes:  {'trafficLightColor': [0, 'NA']}
object:  3  category:  traffic light  attributes:  {'trafficLightColor': [0, 'NA']}
object:  4  category:  traffic light  attributes:  {'trafficLightColor': [1, 'G']}
object:  5  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  6  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  7  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  8  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  9  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  10  category:  traffic sign  attributes:  {'trafficLightColor': [0, 'NA']}
object:  11  category:  car  attrib

In [ ]:
len(data_sub)

4641

## Computation of Set Difference

In [ ]:
actor_scores = []
scene_scores = []
for i in range(len(data_sub)):
    # get the id for the bdd100k data instance
    vid = data_sub[i]['videoName']
    # get the actors, attributes and scene from bdd100k

    bdd100kActors = []
    bdd100kScene = []
    labels_list = (data_sub[i]['labels'])
    bdd100kScene.append(data_sub[i]['attributes']['scene'])
    if labels_list:
        for i, x in enumerate(labels_list):
            if (x['category'] == 'traffic sign'):
                bdd100kScene.append('signs')
            if (x['category'] == 'traffic light'):
                bdd100kScene.append('light')
                bdd100kScene.append('lights')
                bdd100kScene.append('red light')
                bdd100kScene.append('red traffic light')
                bdd100kScene.append('traffic lights')
                bdd100kScene.append('yellow light')
                bdd100kScene.append('yellow traffic light')
                bdd100kScene.append('green light')
                bdd100kScene.append('green traffic light')
                bdd100kScene.append('traffic signal')
            if (x['category'] == 'bike'):
                bdd100kActors.append('cyclist')
            if (x['category'] == 'bus' or x['category'] == 'truck'):
                bdd100kActors.append('heavy vehicle')
            if (x['category'] == 'car' or x['category'] == 'motor'):
                bdd100kActors.append('light vehicle')
            if (x['category'] == 'person' or x['category'] == 'rider'):
                bdd100kActors.append('pedestrian')

    bdd100kActors_set = set(bdd100kActors)
    bdd100kScene_set = set(bdd100kScene)
    # get the corresponding bddx instance
    bddx_instances = bddx.loc[bddx['BDDX-ids'] == vid]
    if (len(bddx_instances) > 1):
        continue
    else:
        # get the SDL object corresponding to the bddx instance that matches the bdd100k instance we're looking at
        index = bddx_instances.index[0]
        bddxScene = []
        for i in sdlObjectList[index-1].scene.values():
            if len(i) !=0:
                for x in i:
                    bddxScene.append(x)
        bddx_actors = []
        for a in range(len(sdlObjectList[index-1].actors)):
            actorsIndex = str(a+1)
            for j in range(len(sdlObjectList[index-1].actors[actorsIndex])):
                bddx_actors.append(sdlObjectList[index-1].actors[actorsIndex][j].description)
        set_actors = set(bddx_actors)
        set_scene = set(bddxScene)
        if ('ego' in set_actors):
            set_actors.remove('ego')
        if ('NaN' in set_actors):
            set_actors.remove('NaN')
        if ('NaN' in set_scene):
            set_scene.remove('NaN')
        actor_intersection = intersection(list(bdd100kActors_set), list(set_actors))
        scene_intersection = intersection(list(bdd100kScene_set), list(set_scene))
        actor_scores.append(len(set_actors) - len(actor_intersection))
        scene_scores.append(len(scene_intersection))



In [ ]:
sum(actor_scores)/len(actor_scores)

0.27390515677636695

In [ ]:
sum(scene_scores)/len(scene_scores)

0.15133454262762375